In [1]:
import numpy as np
from scipy import sparse
from tables import *
import time
from tqdm import tqdm_notebook as tqdm
import csv

data_path = "/mnt/disks/sec/data/"

In [2]:
h5file = open_file(data_path+"citations.h5", mode="a", title="citations")
table_cits = h5file.root.citations.cits


In [3]:
dict_cits = {}

In [4]:
nb_l = len(table_cits)


In [ ]:
for i in tqdm(range(nb_l)):
    row = table_cits[i]
    cite = row['cite']
    citant = row['citant']
    if citant in dict_cits:
        dict_cits[citant] = np.append(dict_cits[citant],cite)
    else:
        dict_cits[citant] = np.array([cite],dtype=np.uint32)

/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


A Jupyter Widget

In [6]:
h5file.close()

In [7]:
info_path = "/mnt/disks/sdb/data/pub_exp_info.txt"

In [8]:
dict_year = {}
with open(info_path, newline='') as f:
    reader = csv.reader(f,delimiter='\t', quoting=csv.QUOTE_NONE)
    next(reader)
    for line in reader:        
        dict_year[int(line[0])] = int(line[2])

In [9]:
class DictCits(IsDescription):
    citant = UInt32Col()
    cites = StringCol(itemsize=4000, dflt=" ", pos=0)
    year_citant = UInt16Col()


In [10]:
h5file = open_file('/mnt/disks/sdb/data/' + 'DictCits.h5', mode="w", title="DictCits")
group = h5file.create_group("/", 'pub_exp')
table = h5file.create_table(group, 'DictCits', DictCits)
citations = table.row


In [11]:
nb_citants = len(dict_cits)

In [12]:
keys_cits = list(dict_cits.keys())

In [ ]:
for i in tqdm(range(nb_citants)):
    citant = keys_cits[i]
    citations['cites'] = dict_cits[citant].tostring()
    citations['citant'] = citant
    citations['year_citant'] = dict_year[citant]
    citations.append()
table.flush()

/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


A Jupyter Widget

In [ ]:
h5file.close()